# Existential Risk Model Solution with Value Function Iteration

This notebook is part of a computational appendix that accompanies the paper

> Existential Risk and The Fermi Paradox
> > Mahon (2021)

Code is adapted from [7 Solution Methods to Solve the Neoclassical Growth Model with Python](https://notes.quantecon.org/submission/5b5f70db9cd7f00015be634e) from the computational appendix of *MATLAB, Python, Julia: What to Choose in Economics?* by Coleman, Lyon, Maliar, and Maliar (2017). 


In [392]:
import numpy as np
import pandas as pd
import scipy.optimize as opt
import time
import matplotlib.pyplot as plt

from collections import namedtuple
from scipy.interpolate import interp1d
from numba import jit, vectorize

## Model Description

There is a fixed population of N homogeneous agents who live up to an infinite number of periods.

There are two control variables: consumption ($ c $), and existential risk mitigation $( w )$. Agents maximize their expected lifetime utility with respect to these two variables. 

The are two state variables: a general [common-pool resource](https://en.wikipedia.org/wiki/Common-pool_resource) (CPR), $ X $, and [total-factor productivity](https://en.wikipedia.org/wiki/Total_factor_productivity) (TFP), $ A $. Each agent's production function relies on how much of the CPR they extract $ x_i $, multiplied by $ A $.   
$$
\begin{equation}
    f(x_i, A) = Ax_i \tag{production function}
\end{equation}
$$    
    
Thus, we have the following for the budget constraint: 
$$ 
\begin{align}
    c_i + w_i &\leq Ax_i \tag{B.C.} \\
    \sum_{i=1}^N x_i &\leq X \\
    \implies C + W &\leq AX; \;\; \text{ where } C \equiv \sum_{i=1}^N c_i, \text{ and } W \equiv \sum_{i=1}^N w_i
\end{align}
$$
Whatever CPR remains at the end of a period goes to the next period multiplied by an exogenous rate of replenishment $ g_x $. Using the notation that primes mean a variables value in the next period, we get the following for the law of motion for the CPR. 
$$
\begin{align}
    X' &= ( X - \sum_{i=1}^N x_i )(1 + g_x) \tag{CPR LLM }\\
    &= (X - \frac{C + W}{A})( 1 + g_x )
\end{align}
$$
     
The TFP grows at a constant exponential rate, $ g_a $:
$$
\begin{equation}
    A' = A(1 + g_a)
\end{equation}
$$  

We can combine these last two equations by multiplying the CPR LLM by $  A(1 + g_a) $:
$$
\begin{equation}
    A'X' = (AX - C - W)( 1 + g_x )(1 + g_a) 
\end{equation}
$$ 
    
To simply, denote $ \tilde{X} := A'X' $ and $ g := g_x + g_a + g_x g_a $, such that $ ( 1 + g) = ( 1 + g_x)(1 + g_a) $. Now the problem can be expressed in terms of a single state variable, $ \tilde{X} $. 
$$
\begin{align}
    \tilde{X}' &= (\tilde{X} - C - W)( 1 + g) \\
    C + W &\leq \tilde{X} 
\end{align}
$$ 


The instantaneous utility is a log-utility functon, and we assume the utility of death is zero, i.e. agents are indifferent to being dead and consuming just 1 unit of consumption because $\ln(1) = 0 $, and they prefer death over $ c < 1 $, and prefer life over $ c > 1 $. Future utility is discounted by the subjective discount factor, $ \beta \in [0,1) $. 

There is endogenous probability of civilization ending due to an existential disaster at the end of each period based on the ratio of the current period's total consumption, $ C $, and total existential risk mitigation, $ W $. The scalar constant, $ \bar{a} $, scales the relative effect of $ W $ versus $ C $ on this probability. 
$$
\Pr(\text{ Existiential Disaster } | C, W ) = \frac{C}{C + \bar{a} W}
$$

The agent's problem can be written recursively using the following Bellman equation:

$$
\begin{align*}
    V(\tilde{X}) &= \max_{c_i,w_i} \biggl\{\ln(c_i) + \beta E \bigl[V(\tilde{X}') | C, W\bigr] \biggr\}\\
    & \text{ s.t. } c_i \geq 0; \qquad s_i \geq 0 \\
    & \phantom{\text{ s.t. } } C + W \leq \tilde{X}\\
    & \phantom{\text{ s.t. } } \tilde{X}' = (\tilde{X} - C - W)(1 + g) 
\end{align*} 
$$

Note:
$$
\begin{align}
    E \bigl[V(\tilde{X}') | C, W\bigr] &= \bigg( 1 - \Pr(\text{ Existiential Disaster } | C, W )\bigg)V(\tilde{X}') \\
    &= \frac{\bar{a}W}{C + \bar{a} W}V(\tilde{X}') 
\end{align} 
$$
    
Thus, the problem can be expressed deterministically:
$$
\begin{align*}
    V(\tilde{X}) &= \max_{c_i,w_i} \biggl\{\ln(c_i) + \beta \frac{\bar{a}W}{C + \bar{a} W} V(\tilde{X}')\biggr\}\\
    & \text{ s.t. } c_i \geq 0; \qquad s_i \geq 0 \\
    & \phantom{\text{ s.t. } } C + W \leq \tilde{X}\\
    & \phantom{\text{ s.t. } } \tilde{X}' = (\tilde{X} - C - W)(1 + g) 
\end{align*} 
$$

## Python Code

We begin by defining a `namedtuple` that contains the parameters of our model. This is useful to pass the parameters around to functions that are just-in-time (JIT) compiled by `numba`.

In [393]:
#
# Create a named tuple type that we can pass into the jitted functions
# so that we don't have to pass parameters one by one
#
Params = namedtuple("Params", ["N", "beta", "g", "a_bar"])

@jit(nopython=True)
def param_unpack(params):
    "Unpack parameters from the Params type"
    out = (params.N, params.beta, params.g,
           params.a_bar)

    return out

We will then define various helper functions to ensure that we [don't repeat ourselves](https://lectures.quantecon.org/py/writing_good_code.html#don-t-repeat-yourself) and that the inner functions can be JIT compiled.

In [394]:
#
# Helper functions to make sure things are jitted
#
@vectorize(nopython=True)
def u(c):
    "log utility function"
    return -1e10 if c < 1e-10 else np.log(c)

@vectorize(nopython=True)
def psurvival(a_bar, c, w):
    return a_bar*w/(c + a_bar*w)


## Envelope Condition 

Denote the optimal $ \tilde{X}' $ as $ \tilde{X}^{\prime \ast} $ and the optimal $ c $ and $ w $ as $ c^\ast $ and $ w^\ast $. 

$ \tilde{X}^{\prime \ast}$ is defined as the following:
$$ \begin{equation}
\tilde{X}^{\prime \ast} \equiv (\tilde{X} - C(c^\ast) - W(w^\ast))(1 + g)
\end{equation}
$$
With total consumption $ C $ and total risk reduction investment $ W $ expressed as functions of the representative agents choices. 

$ W(w^\ast) $ can now be expressed as:
$$ \begin{equation}
W(w^\ast) \equiv \tilde{X}  - \frac{\tilde{X}^{\prime \ast}}{1 + g} - C(c^\ast)
\end{equation}
$$
Thus, fixing $ w $ and $ \tilde{X}' $ at the optimum, the problem can be described with consumption as the only control variable:
$$
\begin{align*}
    V(\tilde{X}) &= \max_{c} \biggl\{\ln(c) + \beta \frac{\bar{a}\bigg( \tilde{X} - \frac{\tilde{X}^{\prime \ast} }{1 + g} - C(c)\bigg)}{C(c) + \bar{a} \bigg( \tilde{X} - \frac{\tilde{X}^{\prime \ast} }{1 + g} - C(c) \bigg)} V(\tilde{X}^{\prime\ast} )\biggr\}
\end{align*} 
$$
We can now solve for $ c^\ast $ using the FOC:
$$
\begin{align}
c^\ast &= - \Bigg\{ \beta V(\tilde{X}^{\prime \ast}) \frac{d}{dc}\Bigg(\frac{\bar{a}\bigg( \tilde{X} - \frac{\tilde{X}^{\prime \ast} }{1 + g} - C(c)\bigg)}{C(c) + \bar{a} \bigg( \tilde{X} - \frac{\tilde{X}^{\prime \ast} }{1 + g} - C(c) \bigg)} \Bigg) \Bigg\}^{-1}
\end{align}
$$

The envelope condition tells us that at the optimum, $ V(\tilde{X}') $ must equal $ V(\tilde{X}^{\prime \ast}) $. Thus, changes in the optimizers $ c $ and $ w $ will not change $ V(\tilde{X}^{\prime \ast}) $ in a sense because it is already defined as being at the optimum.    the value function can be written as:
$$
\begin{align*}
    V(\tilde{X}) &= \max_{c_i,w_i} \biggl\{\ln(c_i) + \beta \frac{\bar{a}W(w_i)}{C(c_i) + \bar{a} W(w_i)} V(\tilde{X}^{\prime\ast} )\biggr\}
\end{align*} 
$$

#### Social Optimum
At the social optimum, we maximize w.r.t. to the same $ c $ for everyone so, $ C(c) = Nc $. Thus, at the social optimum we get the following for $ c^\ast $ from the first order condition.
$$ 
\begin{align*}
    c^\ast &=  \frac{\bar{a} (X(1 + g) - \tilde{X}^{\prime \ast} ) \bigg(2 (\bar{a} - 1) + \beta V(\tilde{X}^{\prime \ast})  - \sqrt{\beta V(\tilde{X}^{\prime \ast})  \bigg(\beta V(\tilde{X}^{\prime \ast}) + 4 (\bar{a} - 1 )\bigg)}\bigg)}{2 N (\bar{a} - 1)^2 (1 + g)}
\end{align*}
$$

In [395]:
@vectorize(nopython=True)
def so_foc_c(X, Xp, Vp, a_bar, beta, N, g):
    """
    Social optimum c expressed in terms of optimal X' from first order condition.
    Ensure values can not be negative nor exceed X/N
    """
    return (
            a_bar*(X*(1 + g) - Xp)*(
                2*(a_bar - 1) + beta*Vp - (
                    beta*Vp*(
                        beta*Vp + 4*(a_bar - 1)
                    )
                )**0.5
            )
        )/(
            2*N*((a_bar - 1)**2)*(1 + g)
        )

@vectorize(nopython=True)
def so_foc_w(X, Xp, N, g, c):
    "Social optimum w expressed in terms of optimal X' and c from first order condition"
    return (X - Xp/(1 + g) - N*c)/N



#### Market Equilibrium
At the market equilibriums agent's maximize their own individual utility with respect to their own consumption values for $ c $ and $ w $. They optimize against the other agents' choices, which we donote as $ \bar{c} $. All other agents must make the same choice since this is a homogeneous agent model, thus $ C(c) $ becomes $ c + (N-1)\bar{c} $, and now we can use the same process to find $ c^\ast $ in terms of the optimal $ \tilde{X}^{\prime \ast } $ and the model conditions, which include the strategies of the other agents. The result to this FOC is much longer than the one for the social optimum, so rather than simplifying it, the solution, which was computed in MATLAB, has been adapted to the Python function shown below:

In [396]:
@vectorize(nopython=True)
def me_foc_c(X, Xp, Vp, a_bar, beta, N, g, c_bar):
    "Market Equilibrium c expressed in terms of optimal X' from first order condition"
    return (
        2*c_bar - (
          Vp*a_bar*beta*(X - Xp + X*g)*(
            4*c_bar 
            - 4*N*c_bar 
            - 4*X*a_bar 
            + 4*Xp*a_bar 
            - 8*a_bar*c_bar 
            + 4*c_bar*g 
            + 4*X*a_bar**2 
            - 4*Xp*a_bar**2 
            + 4*a_bar**2*c_bar 
            + 8*N*a_bar*c_bar 
            - 4*N*c_bar*g 
            - 4*X*a_bar*g 
            - 8*a_bar*c_bar*g 
            - 4*N*a_bar**2*c_bar 
            + 4*X*a_bar**2*g 
            + 4*a_bar**2*c_bar*g 
            - 4*N*a_bar**2*c_bar*g 
            + Vp*X*a_bar*beta 
            - Vp*Xp*a_bar*beta 
            + 8*N*a_bar*c_bar*g 
            + Vp*X*a_bar*beta*g
            )
        )**(1/2) 
        - 2*N*c_bar 
        - 2*X*a_bar 
        + 2*Xp*a_bar 
        - 4*a_bar*c_bar 
        + 2*c_bar*g 
        + 2*X*a_bar**2 
        - 2*Xp*a_bar**2 
        + 2*a_bar**2*c_bar 
        + 4*N*a_bar*c_bar 
        - 2*N*c_bar*g 
        - 2*X*a_bar*g 
        - 4*a_bar*c_bar*g 
        - 2*N*a_bar**2*c_bar 
        + 2*X*a_bar**2*g 
        + 2*a_bar**2*c_bar*g 
        - 2*N*a_bar**2*c_bar*g 
        + Vp*X*a_bar*beta 
        - Vp*Xp*a_bar*beta 
        + 4*N*a_bar*c_bar*g 
        + Vp*X*a_bar*beta*g
      )/(
        2*(
          g - 2*a_bar - 2*a_bar*g + a_bar**2*g + a_bar**2 + 1
        )
      )

@vectorize(nopython=True)
def me_foc_w(X, Xp, N, g, c_bar, w_bar, c):
    "Market Eq'm w expressed in terms of optimal X' and c from first order condition"
    return X - Xp/(1 + g) - (N-1)*(c_bar + w_bar) - c


### The Policy and Value Function 

Now that we can derive $ c^\ast $ and $ w^\ast $ from $ \tilde{X}^{\prime \ast } $ and $ V(\tilde{X}^{\prime \ast }) $, we just need to find functions for these two values in terms of the current state $ \tilde{X} $. A policy function: $ \widehat{\tilde{X}^{\prime \ast } }(\tilde{X})$, and a value function: $ \widehat{V}(\tilde{X}) $. 

However, traditional approaches to this particular model will not work because until the model is solved, we can not know how many periods this economy will survive until. Given some initial conditions the equilibrium may be for the CPR to potentially stay positive forever. Other conditions could lead to the CPR being entirely depleted in one period, or depleted in $ n $ periods from the present. Thus, this model has an endogenous time horizon, $ \in [0, \infty) $. This means that no analytical function could fully express these solutions on their, and because the state could also grow infinitely, traditional non-analytic approaches, such as spline functions will do a poor job of extrapolating values outside of the fitted range. 

This leads us to the following two-part approach. We will use a linear spline for low state values, $ \tilde{X} $, where there is a lot of discontinuity, and an analytic function for high $ \tilde{X} $, which could competently extrapolate estimates for $ \tilde{X} $ higher than what it was fitted on.

Through trial and error, we find that on the analytic portion of the domain for these functions, $ \widehat{\tilde{X}^{\prime \ast } }(\tilde{X}; b) $ follows a simple linear equation:
$$
\begin{align}
 \widehat{\tilde{X}^{\prime \ast } }(\tilde{X}; b) = b_0 + b_1\tilde{X}
\end{align}
$$

And $ \widehat{V}(\tilde{X}; b) $ can be estimated with the following nonlinear functional form:
$$
\widehat{V}(\tilde{X}; b) = b_0 + b_1 \frac{\ln(\tilde{X})}{b_2 - \tilde{X}^{b_3}}
$$

Their Python functions are shown below. 

In [397]:
def x_curve(x, b0, b1):
    "X' policy function curve"
    return b0 + b1*x


def v_curve(x,  b0, b1, b2, b3):
    "Value function curve"
    return b0 + b1*np.log(x)/(b2 - (x**b3))

We also now define a class that contains

1. Parameters of the model
2. A grid used for approximating the solution

This again helps us maintain all of the relevant information in one place and simplifies passing it to other functions.

In [398]:
class ExistentialRiskModel(object):
    """
    * N: Population Size
    * beta: subjective discount factor
    * g_x: CPR regrowth rate
    * g_a: TFP growth rate
    * a_bar: Exisitentail risk reduction scaling parameter
    """
    def __init__(self, N=1, beta=0.95, g_x=0.05, g_a=0.015, a_bar=1e5, 
                 xmin_l=2, xmax_l=200, nx_l=25, linspace_l=True,
                 xmin_h=250, xmax_h=2500, nx_h=100, linspace_h=False):
        # calculate g from g_a and g_x 
        g = g_a + g_x + g_a*g_x

        # Household parameters
        self.beta = beta

        # Economy parameters
        self.N, self.g, self.a_bar = N, g, a_bar

        # Create x grid for spline interpolation domain
        # xmin and xmax refer to the X minimum and maximum per agent, so they must be scaled up by N.
        if linspace_l:
            self.xgrid_l = N*np.linspace(xmin_l, xmax_l, nx_l)
        else:
            self.xgrid_l = N*np.geomspace(xmin_l, xmax_l, nx_l)
        
        # Create x grid for analytic function domain
        if linspace_h:
            self.xgrid_h = N*np.linspace(xmin_h, xmax_h, nx_h)
        else:
            self.xgrid_h = N*np.geomspace(xmin_h, xmax_h, nx_h)
            
        # concatenate into a single grid
        self.xgrid = np.concatenate([self.xgrid_l, self.xgrid_h])
        # store the state value that splits the two parts of the solution and its index in the xgrid
        self.xsplit = self.xgrid_l[-1]
        self.xsplit_idx = nx_l
        
        self.ns = nx_l + nx_h
        
    def _unpack_params(self):
        out = (self.N, self.beta, self.g, self.a_bar)
        return out

    def _unpack_grids(self):
        out = (self.xgrid)
        return out


In [399]:
class GeneralSolution:
    """
    This is a general solution method. We define this, so that we can
    sub-class it and define specific update methods for each particular
    solution method
    """
    def __init__(self, erm, prev_sol=None, maxfev=10000, interpolator='linear', xsplit=None):
        # Save model and approximation degree
        self.erm,  self.maxfev, self.interpolator = erm, maxfev, interpolator

        # Unpack some info from erm
        N, beta, g, a_bar = self._unpack_params()
        X = self.erm.xgrid
        if xsplit is None:
            self.xsplit = self.erm.xsplit
            self.xsplit_idx = self.erm.xsplit_idx
        else:
            self.xsplit = xsplit
            self.xsplit_idx = np.searchsorted(X, xsplit, side='right')
        

        # Use parameter values from model to create a namedtuple with
        # parameters saved inside
        self.params = Params(N, beta, g, a_bar)

        # Update to fill initial value and policy matrices
        # If we give it another solution type then use it to
        # generate values and policies
        if issubclass(type(prev_sol), GeneralSolution):
            self.XP = prev_sol.XP
            self.VF = prev_sol.VF
        else:
            # guess VF and XP
            self.VF = np.log(X/N) 
            self.XP = X

            
        # Fit spline based on guesses
        self.x_spline = interp1d(X[:self.xsplit_idx]
                                 , self.XP[:self.xsplit_idx]
                                 , kind=self.interpolator
                                 , fill_value='extrapolate')
        self.v_spline = interp1d(X[:self.xsplit_idx]
                                 , self.VF[:self.xsplit_idx]
                                 , kind=self.interpolator
                                 , fill_value='extrapolate')
        
        # Fit coefficients of analytics functions based on guesses
        self.x_coeffs, _ = opt.curve_fit(x_curve
                                         , X[self.xsplit_idx:]
                                         , self.XP[self.xsplit_idx:]
                                         , maxfev=self.maxfev)
        self.v_coeffs, _ = opt.curve_fit(v_curve
                                         , X[self.xsplit_idx:]
                                         , self.VF[self.xsplit_idx:]
                                         , maxfev=self.maxfev)
    
    def build_VF(self, X=None):
        """
        Using the current coefficients, this builds the value function
        for all states
        """
        if X is None:
            X = self.erm.xgrid
        
        VF = np.maximum(0, np.where(X > self.xsplit
                        , v_curve(X, *self.v_coeffs)
                        , self.v_spline(X)
                    ))
                

        return VF
    
    def build_XP(self, X=None):
        if X is None:
            X = self.erm.xgrid
        
        XP = np.where(X > self.xsplit
                        , np.clip(
                                x_curve(X, *self.x_coeffs)
                                , 0, X*(1 + self.erm.g)
                            )
                        , np.clip(
                            self.x_spline(X)
                            , 0, X*(1 + self.erm.g)
                        )
                     )
        
        return XP
    
    
    def compute_coeffs(self, XP, VF):
        if isinstance(self, IterateOnPolicy):
            new_x_coeffs = self.x_coeffs
        else:
            new_x_coeffs, _ = opt.curve_fit(x_curve
                                            , self.erm.xgrid[self.xsplit_idx:]
                                            , XP[self.xsplit_idx:]
                                            , maxfev=self.maxfev)
        
        new_v_coeffs, _ = opt.curve_fit(v_curve
                                        , self.erm.xgrid[self.xsplit_idx:]
                                        , VF[self.xsplit_idx:]
                                        , maxfev=self.maxfev) 
        
        return new_x_coeffs, new_v_coeffs
    
    def compute_splines(self, XP, VF):
        if isinstance(self, IterateOnPolicy):
            new_x_spline = self.x_spline
        else:
            new_x_spline = interp1d(self.erm.xgrid[:self.xsplit_idx]
                                    , XP[:self.xsplit_idx]
                                    , kind=self.interpolator
                                    , fill_value='extrapolate')
        
        new_v_spline = interp1d(self.erm.xgrid[:self.xsplit_idx]
                                , VF[:self.xsplit_idx]
                                , kind=self.interpolator
                                , fill_value='extrapolate')
        
        return new_x_spline, new_v_spline

    def _unpack_params(self):
        return self.erm._unpack_params()

    def update_v(self, new_v_coeffs, new_v_spline):
        """
        Updates the coefficients for the value function
        """
        self.v_coeffs = new_v_coeffs
        self.v_spline = new_v_spline

        return None

    def update_x(self, new_x_coeffs, new_x_spline):
        """
        Updates the coefficients for the policy function
        """
        self.x_coeffs = new_x_coeffs
        self.x_spline = new_x_spline
        
        return None
    

    def compute_distance(self, XP, VF):
        """
        Computes average distance between policy functions
        """
        return np.max(np.abs(1.0 - (XP+1e-12)/(self.XP+1e-12)))
    

    def solve(self, tol=1e-6, maxiter=2500, verbose=False, nskipprint=25):
        # Iterate until convergence
        dist, it = 10.0, 0
        while (dist>tol) and (it<maxiter):
            # Run solution specific update code
            XP, VF = self.update()
            

            # Compute new policy and value coeffs
            new_x_coeffs,  new_v_coeffs = self.compute_coeffs(XP, VF)
            new_x_spline, new_v_spline = self.compute_splines(XP, VF)

            # Update distance and iterations
            dist = self.compute_distance(XP, VF)
            it += 1
            if verbose and it%nskipprint == 0:
                print(it, dist)
                print('max dist idx: ', np.argmax(np.abs(1.0 - (XP+1e-12)/(self.XP+1e-12))))
            
            # update policy and value functions
            self.XP, self.VF = XP, VF
            
            # Update functions params
            self.update_x(new_x_coeffs, new_x_spline)
            self.update_v(new_v_coeffs, new_v_spline)

        # After finishing iteration, iterate to convergence using policy
        if not isinstance(self, IterateOnPolicy):
            sol_iop = IterateOnPolicy(self.erm, prev_sol=self, maxfev=self.maxfev)
            XP, VF = sol_iop.solve(tol=1e-6)

            # Save final versions of everything
            self.XP, self.VF = XP, VF
            
            new_x_coeffs, new_v_coeffs = sol_iop.compute_coeffs(XP, VF)
            new_x_spline, new_v_spline = sol_iop.compute_splines(XP, VF)
            self.update_x(new_x_coeffs, new_x_spline)
            self.update_v(new_v_coeffs, new_v_spline)

        return self.XP, self.VF
    
    """
    This is where we run the Bellman operator on the policy and value functions.
    Updating their new values before fitting new functions on each iteration.
    """
    def update(self):
        # Unpack parameters
        N, beta, g, a_bar = self._unpack_params()
        xgrid = self.erm.xgrid
        n_state = xgrid.shape[0]

        # Get the policy and update it
        XP = np.empty(n_state)
        VF = np.empty(n_state)
        for i_s in range(n_state):
            # Pull out current state
            x = xgrid[i_s]        
            
            _xp, fval, _, _ = opt.fminbound(lambda _xp: -self.compute_V(X=x, XP=_xp)
                        , 0
                        , x*(1+g)
                        , xtol=1e-12
                        , full_output=True
                    )
            _vf = -fval
            
            vf_corner = self.compute_V(X=x, XP=0)
            if (_vf <= vf_corner):
                _xp = 0
                _vf = vf_corner

            XP[i_s] = _xp
            VF[i_s] = _vf 

        return XP, VF
        


In [400]:
class Social_Optimum(GeneralSolution):
    """
    Updates the coefficients and value functions using the VFI
    method
    """
    def build_CP(self, X=None, XP=None, Vp=None):
        N, beta, g, a_bar = self._unpack_params()
        
        if X is None:
            X = self.erm.xgrid
        
        if XP is None:
            XP = self.build_XP(X)
            
        if Vp is None:
            Vp = self.build_VF(XP)
        
        exp = np.maximum(0, X - XP/(1+g))
            
        return np.clip(so_foc_c(X, XP, Vp, a_bar, beta, N, g), 0, exp/N)
    
    def build_WP(self, X=None, XP=None, CP=None):
        N, beta, g, a_bar = self._unpack_params()
        
        if X is None:
            X = self.erm.xgrid
        
        if XP is None:
            XP = self.build_XP(X)
            
        if CP is None:
            CP = self.build_CP(X=X, XP=XP)
        
        return so_foc_w(X, XP, N, g, CP)
    
    def compute_V(self, X=None, XP=None, Vp=None, CP=None, WP=None):
        # Unpack parameters
        N, beta, g, a_bar = self._unpack_params()
        
        if Vp is not None and CP is not None and WP is not None:
            VF = u(CP) + beta*psurvival(a_bar, CP, WP)*Vp
            
        else:
            if X is None:
                X = self.erm.xgrid
                
            if XP is None:
                XP = self.build_XP(X)
                
            if Vp is None:
                Vp = self.build_VF(XP)
                
            if CP is None:
                CP = self.build_CP(X=X, XP=XP, Vp=Vp) 
                
            if WP is None:
                WP = self.build_WP(X=X,XP=XP,CP=CP) 
                
            VF = u(CP) + beta*psurvival(a_bar, CP, WP)*Vp
            
        return np.maximum(0, VF)

    
    def simulate(self, X=None, T=1000, s=1):
        """
        Simulates the existential risk model with policy function
        given by self.XP. Run Simulation for T periods, and use s 
        to scale policy function estimates to check how well model 
        performs compared to slight modifications. The simulation results
        should ideally be highest when s=1. 
        """
        N, beta, g, a_bar = self._unpack_params()
        # X is starting point, if None, set to N*274
        if X is None:
            X = N*274
            
        def recurse_bellman(x, t):
            if x <= 0:
                return 0, 0, x, t
            if t >= min(T, 2000):
                return u(x/N), 1, x, t
            c_corner = x/N 
            u_corner = u(c_corner)
            if u_corner <= 0:
                return 0, 0, x, t
            
            xp = self.build_XP(x)
            if xp <= 0:
                return u_corner, 0, 0, t+1
            if s != 1:
                xp = s*xp
            xp = min(xp, x*(1+g)-1e-10) # xp = x*(1+g), would result in c = 0, log(0)= -infinity in current period
            
            vp_hat = self.build_VF(xp)
            c = np.clip(self.build_CP(X=x,XP=xp,Vp=vp_hat)
                        ,1e-10,c_corner)
            w = self.build_WP(X=x, XP=xp, CP=c)
            
            p_t = psurvival(a_bar, c, w)
            u_t = u(c)
            
            vp, pp, final_state, nperiods = recurse_bellman(xp, t+1)            
            
            if u_t + beta*p_t*vp <= u_corner:
                return u_corner, 0, 0, t+1
            
            return u_t + beta*p_t*vp, p_t*pp, final_state, nperiods
        
        T_counter = 0 
        psurvived = 1
        total_VP = 0
        while T > 0:
            vp, pp, X, nperiods = recurse_bellman(X, 0)
            total_VP += beta**(T_counter)*psurvived*vp
            psurvived *= pp
            T_counter += nperiods
            T -= 2000
            
        return total_VP, psurvived, X, T_counter

### Iterating to Convergence (given policy)

This isn't one of the methods described above, but it is used as an element of a few of our methods (and also as a way to get a first guess at the value function). This method takes an initial policy function, $\hat{\tilde{X}^{\prime\ast}}(\tilde{X})$, as given, and then, without changing the policy, iterates until the value function has converged.

Thus the "update section" of the algorithm in this instance would be:

* Leave policy function unchanged
* At each point of grid, $(\tilde{X})$, compute $\hat{V}(\tilde{X}) = u\bigg(c^\ast\big(\hat{\tilde{X}^{\prime\ast}}(\tilde{X})
\big)\bigg) + \beta E \left[ V(\hat{\tilde{X}^{\prime\ast}}(\tilde{X})) \right]$

We override the methods from `GeneralSolution`

* `compute_distance` because when we are iterating to convergence on the value function we want to check distnace using value function rather than policy function


The `update` method just repeatedly applies a particular policy function to update the value function.

In [401]:
class IterateOnPolicy(Social_Optimum):
    """
    Subclass of the general solution method. The update method for this
    class simply computes the fixed point of the value function given
    a specific policy
    """
    def compute_distance(self, XP, VF):
        """
        Computes distance between policy functions. When we are
        iterating on a specific policy, we would like to compute
        distances by the difference between VFs
        """
        return np.max(np.abs(1.0 - (VF )/(self.VF )))
    
    def update(self):
        VF = self.compute_V()
        
        return self.XP, VF

## Run Solution

To start we can initialize the model

In [402]:
erm = ExistentialRiskModel(N=2, beta=0.95, g_x=0.05, g_a=0.015, a_bar=1e5, 
                 xmin_l=2, xmax_l=100, nx_l=25, linspace_l=True,
                 xmin_h=110, xmax_h=3000, nx_h=100, linspace_h=False
            )

Now we can iterate over the policy function guess to fit a value function

In [403]:
# First guess
vp = IterateOnPolicy(erm, maxfev=50000, interpolator='linear')
vp.solve(tol=1e-9)
print('vp done')

/Users/mikemahon/anaconda3/lib/python3.8/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/Users/mikemahon/anaconda3/lib/python3.8/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


AttributeError: 'IterateOnPolicy' object has no attribute 'x_spline'

### Solving for Social Optimum 

Pass the value function result from the initial guess into the Social_Optimum solution class. Then call .solve() to perform value function iteration until we find a fixed point where the value function and policy functions converge

In [ ]:
so_sol = Social_Optimum(erm, maxfev=100000, prev_sol=vp, interpolator='linear')
ts = time.time()
so_sol.solve(tol=1e-6, verbose=True, nskipprint=25)
time_took = time.time() - ts
print(time_took)

In [ ]:
X = so_sol.erm.xgrid
XP = so_sol.XP
xmax = so_sol.erm.xgrid[-1] +1
xnew = np.linspace(1,xmax,100)
xp_new = so_sol.build_XP(xnew)

plt.plot(X[X < xmax], XP[X < xmax], 'o', xnew, xp_new)
plt.xlabel('X')
plt.ylabel('XP')
plt.title("XP")

## Market Equilibrium

In [ ]:
class MarketEquilibrium(GeneralSolution):
    """
    Updates the coefficients and value functions using the VFI
    method
    """
    def __init__(self, *args, prev_sol_bar=None, **kwargs):
        super().__init__(*args, **kwargs)
        N, beta, g, a_bar = self._unpack_params()
        
        if issubclass(type(prev_sol_bar), GeneralSolution):
            self.prev_sol_bar = prev_sol_bar
        else:
            self.prev_sol_bar = kwargs['prev_sol'].prev_sol_bar
        
        self.CPbar = self.build_CPbar(X=self.erm.xgrid, XPbar=self.prev_sol_bar.XP)
        self.WPbar = self.prev_sol_bar.build_WP(X=self.erm.xgrid, XP=self.prev_sol_bar.XP, CP=self.CPbar)

    def build_CP(self, X=None, XP=None, Vp=None, CPbar=None, WPbar=None,exp=None):
        N, beta, g, a_bar = self._unpack_params()
        
        if X is None:
            X = self.erm.xgrid
            if CPbar is None:
                CPbar = self.CPbar
            if WPbar is None:
                WPbar = self.WPbar
        
        if XP is None:
            XP = self.build_XP(X)
            
        if Vp is None:
            Vp = self.build_VF(XP)
            
        if CPbar is None:
            CPbar = self.build_CPbar(X=X)
            
        if WPbar is None:
            WPbar = self.prev_sol_bar.build_WP(X=X, CP=CPbar)
        
        # expendables
        if exp is None:
            exp = np.maximum(0, X - XP/(1+g) - (N-1)*(CPbar + WPbar))
                
            
        return np.clip(
                np.nan_to_num(me_foc_c(X, XP, Vp, a_bar, beta, N, g, CPbar), nan=1e-10)
                , 0, exp
            )
    
    def build_WP(self, X=None, XP=None, CP=None, CPbar=None, WPbar=None):
        N, beta, g, a_bar = self._unpack_params()
        
        if X is None:
            X = self.erm.xgrid
            if CPbar is None:
                CPbar = self.CPbar
            if WPbar is None:
                WPbar = self.WPbar
                
        if XP is None:
            XP = self.build_XP(X)
        
        if CPbar is None:
            CPbar = self.build_CPbar(X=X)
        
        if WPbar is None:
            WPbar = self.prev_sol_bar.build_WP(X=X, CP=CPbar)
            
        if CP is None:
            CP = self.build_CP(X=X, XP=XP, CPbar=CPbar, WPbar=WPbar)
        
        return me_foc_w(X, XP, N, g, CPbar, WPbar, CP)
    
    def build_CPbar(self, X=None, XPbar=None, Vpbar=None):
        # Unpack parameters
        N, beta, g, a_bar = self._unpack_params()
        
        if X is None:
            X = self.erm.xgrid
        
        return np.minimum(
            self.erm.xgrid/N
            , self.prev_sol_bar.build_CP(X=X, XP=XPbar, Vp=Vpbar)
        )
    
    def compute_V(self, X=None, XP=None, Vp=None, CP=None, WP=None, CPbar=None, WPbar=None):
        # Unpack parameters
        N, beta, g, a_bar = self._unpack_params()
        
        if Vp is not None and CP is not None and WP is not None and CPbar is not None and WPbar is not None:
            VF = u(CP) + beta*psurvival(a_bar, CP + (N-1)*CPbar, WP + (N-1)*WPbar)*Vp
            
        else:
            if X is None:
                X = self.erm.xgrid
                if CPbar is None:
                    CPbar = self.CPbar
                if WPbar is None:
                    WPbar = self.WPbar
                
            if XP is None:
                XP = self.build_XP(X)
                
            if Vp is None:
                Vp = self.build_VF(XP)
                
            if CPbar is None:
                CPbar = self.build_CPbar(X=X)

            if WPbar is None:
                WPbar = self.prev_sol_bar.build_WP(X=X, CP=CPbar)
                
            if CP is None:
                CP = self.build_CP(X=X, XP=XP, Vp=Vp, CPbar=CPbar, WPbar=WPbar) 
                
            if WP is None:
                WP = self.build_WP(X=X,XP=XP,CP=CP, CPbar=CPbar, WPbar=WPbar) 
                
            VF = u(CP) + beta*psurvival(a_bar, CP + (N-1)*CPbar, WP + (N-1)*WPbar)*Vp
            
        return np.maximum(0, VF)
    
    def update(self):
        # Unpack parameters
        N, beta, g, a_bar = self._unpack_params()
        xgrid = self.erm.xgrid
        n_state = xgrid.shape[0]

        # Get the policy and update it
        XP = np.empty(n_state)
        VF = np.empty(n_state)
        for i_s in range(n_state):
            # Pull out current state
            x = xgrid[i_s]   
            cp_bar = self.CPbar[i_s]
            wp_bar = self.WPbar[i_s]
            
        
            
            _xp, fval, _, _ = opt.fminbound(lambda _xp: -self.compute_V(X=x, XP=_xp, CPbar=cp_bar, WPbar=wp_bar)
                        , 0
                        , x*(1+g)
                        , xtol=1e-12
                        , full_output=True
                    )
            _vf = -fval
            
            vf_corner = self.compute_V(X=x, XP=0, CPbar=cp_bar, WPbar=wp_bar)
            if (_vf <= vf_corner):
                _xp = 0
                _vf = vf_corner

            XP[i_s] = _xp
            VF[i_s] = _vf 

        return XP, VF
    
    def simulate(self, X=None, T=1000, s=1):
        """
        Simulates the existential risk model with policy function
        given by self.XP. Run Simulation for T periods, and use s 
        to scale policy function estimates to check how well model 
        performs compared to slight modifications. The simulation results
        should ideally be highest when s=1. 
        """
        N, beta, g, a_bar = self._unpack_params()
        # X is starting point, if None, set to N*274
        if X is None:
            X = N*274
            
        def recurse_bellman(x, t):
            if x <= 0:
                return 0, 0, x, t
            if t >= min(T, 2000):
                return u(x/N), 1, x, t
            
            c_bar = self.build_CPbar(X=x)
            if c_bar >= x/N:
                c_bar = x/N
                w_bar = 0
            else:
                w_bar = self.prev_sol_bar.build_WP(X=x, CP=c_bar)
            
            c_corner = x - (N-1)*(c_bar + w_bar)
            u_corner = u(c_corner)
            if u_corner <= 0:
                return 0, 0, x, t
            
            xp = self.build_XP(x)
            if xp <= 0:
                return u_corner, 0, 0, t+1
            if s != 1:
                xp = s*xp
            xp = min(xp, x*(1+g)-1e-10) # xp = x*(1+g), would result in c = 0, log(0)= -infinity in current period
            
            vp_hat = self.build_VF(xp)
            c = np.clip(self.build_CP(X=x,XP=xp,Vp=vp_hat, CPbar=c_bar, WPbar=w_bar)
                        ,1e-10,c_corner)
            w = self.build_WP(X=x, XP=xp, CP=c, CPbar=c_bar, WPbar=w_bar)
            
            p_t = psurvival(a_bar, c + (N-1)*c_bar, w + (N-1)*w_bar)
            u_t = u(c)
            
            vp, pp, final_state, nperiods = recurse_bellman(xp, t+1)            
            
            if u_t + beta*p_t*vp <= u_corner:
                return u_corner, 0, 0, t+1
            
            return u_t + beta*p_t*vp, p_t*pp, final_state, nperiods
        
        T_counter = 0 
        psurvived = 1
        total_VP = 0
        while T > 0:
            vp, pp, X, nperiods = recurse_bellman(X, 0)
            total_VP += beta**T_counter*psurvived*vp
            psurvived *= pp
            T_counter += nperiods
            T -= 2000
            
        return total_VP, psurvived, X, T_counter
    
class IterateOnPolicy(MarketEquilibrium):
    """
    Subclass of the general solution method. The update method for this
    class simply computes the fixed point of the value function given
    a specific policy
    """
    def compute_distance(self, XP, VF):
        """
        Computes distance between policy functions. When we are
        iterating on a specific policy, we would like to compute
        distances by the difference between VFs
        """
        return np.max(np.abs(1.0 - (VF )/(self.VF )))
    
    def update(self):
        VF = self.compute_V()
        
        return self.XP, VF

In [ ]:
solutions = [so_sol]

# print(' sol #1')
# sol = MarketEquilibrium(erm, maxfev=100000, prev_sol=so_sol, interpolator='linear', prev_sol_bar=so_sol)
# ts = time.time()
# sol.solve(tol=1e-6, verbose=True, nskipprint=25)
# time_took = time.time() - ts
# print(time_took)
# solutions.append(sol)


for i in range(5):
    print(' sol #'+ str(i+1))
    sol = MarketEquilibrium(erm, maxfev=100000, prev_sol=solutions[-1], interpolator='linear', prev_sol_bar=solutions[-1])
    ts = time.time()
    sol.solve(tol=1e-6, verbose=True, nskipprint=25, maxiter=500)
    time_took = time.time() - ts
    print(time_took)
    solutions.append(sol)


In [ ]:
for i in range(1, len(solutions)):
    sol = solutions[i]
    prev_sol = solutions[i-1]
    xp_dist = np.max(np.abs(1.0 - (sol.XP+1e-12)/(prev_sol.XP+1e-12)))
    vf_dist = np.max(np.abs(1.0 - (sol.VF )/(prev_sol.VF )))
    print(i, 'XP dist: ', xp_dist)
    print(i, 'VF dist: ', vf_dist)

In [ ]:
x = X/N

for i in range(len(solutions)):
    lab = "dev. sol "+str(i) if i > 0 else 'social opt.'
    plt.plot(x[20:30], solutions[i].XP[20:30], label = lab)
sol = solutions[-1]
plt.plot(x[20:30], x_curve(N*x[20:30],sol.x_coeffs[0], sol.x_coeffs[1]), label='x curve sol 5')
plt.plot(x[20:30], sol.build_XP(X=N*x[20:30]), label='build_XP sol 5')
plt.legend()
plt.title('XP')
plt.show()

In [ ]:
for i in range(len(solutions)):
    lab = "dev. sol "+str(i) if i > 0 else 'social opt.'
    y = solutions[i].build_CP(XP=solutions[i].XP)
    plt.plot(x[20:30], y[20:30], label = lab)
plt.legend()
plt.title('CP')
plt.show()

In [ ]:
for i in range(len(solutions)):
    lab = "dev. sol "+str(i) if i > 0 else 'social opt.'
    y = solutions[i].build_WP(XP=solutions[i].XP)
    plt.plot(x[20:30], y[20:30], label = lab)
plt.legend()
plt.title('WP')
plt.show()

In [ ]:
sol = solutions[-1] 
# sol = so_sol

In [ ]:
N, beta, g, a_bar = sol._unpack_params()
X = sol.erm.xgrid
VF = sol.VF
XP = sol.XP
Vp = sol.build_VF(XP)
CPbar = sol.CPbar
WPbar = sol.WPbar
CP = sol.build_CP(X=X, XP=XP, Vp=Vp, CPbar=CPbar, WPbar=WPbar)
WP = sol.build_WP(X=X, XP=XP, CP=CP, CPbar=CPbar, WPbar=WPbar)
# CP = sol.build_CP(X=X, XP=XP, Vp=Vp)
# WP = sol.build_WP(X=X, XP=XP, CP=CP)

In [ ]:
sol_df = pd.DataFrame({
    'X': X,
    'X/N': X/N,
    'XP': XP,
    'VF': VF,
    'CP': CP,
    'WP': WP,
    'CPbar': CPbar,
    'WPbar': WPbar,
    'CP + WP': CP + WP,
    'amt. spent': CP + WP + (N-1)*(CPbar + WPbar),
    'CP/WP': CP/WP,
    'X_dot': XP-X,
    'X_dot/X': (XP-X)/X,
})
sol_df.style

In [ ]:
sol = solutions[-2] 
# sol = so_sol

In [ ]:
N, beta, g, a_bar = sol._unpack_params()
X = sol.erm.xgrid
VF = sol.VF
XP = sol.XP
Vp = sol.build_VF(XP)
CPbar = sol.CPbar
WPbar = sol.WPbar
CP = sol.build_CP(X=X, XP=XP, Vp=Vp, CPbar=CPbar, WPbar=WPbar)
WP = sol.build_WP(X=X, XP=XP, CP=CP, CPbar=CPbar, WPbar=WPbar)
# CP = sol.build_CP(X=X, XP=XP, Vp=Vp)
# WP = sol.build_WP(X=X, XP=XP, CP=CP)

In [ ]:
sol_df = pd.DataFrame({
    'X': X,
    'X/N': X/N,
    'XP': XP,
    'VF': VF,
    'CP': CP,
    'WP': WP,
    'CPbar': CPbar,
    'WPbar': WPbar,
    'CP + WP': CP + WP,
    'amt. spent': CP + WP + (N-1)*(CPbar + WPbar),
    'CP/WP': CP/WP,
    'X_dot': XP-X,
    'X_dot/X': (XP-X)/X,
})
sol_df.style

In [ ]:
sol = solutions[7] 
# sol = so_sol

In [ ]:
N, beta, g, a_bar = sol._unpack_params()
X = sol.erm.xgrid
VF = sol.VF
XP = sol.XP
Vp = sol.build_VF(XP)
CPbar = sol.CPbar
WPbar = sol.WPbar
CP = sol.build_CP(X=X, XP=XP, Vp=Vp, CPbar=CPbar, WPbar=WPbar)
WP = sol.build_WP(X=X, XP=XP, CP=CP, CPbar=CPbar, WPbar=WPbar)
# CP = sol.build_CP(X=X, XP=XP, Vp=Vp)
# WP = sol.build_WP(X=X, XP=XP, CP=CP)

In [ ]:
sol_df = pd.DataFrame({
    'X': X,
    'X/N': X/N,
    'XP': XP,
    'VF': VF,
    'CP': CP,
    'WP': WP,
    'CPbar': CPbar,
    'WPbar': WPbar,
    'CP + WP': CP + WP,
    'amt. spent': CP + WP + (N-1)*(CPbar + WPbar),
    'CP/WP': CP/WP,
    'X_dot': XP-X,
    'X_dot/X': (XP-X)/X,
})
sol_df.style

In [ ]:
[sol.build_XP(X=444.731632), sol.build_XP(X=456.950764)]

In [ ]:
sol_df.loc[29]['XP']

In [ ]:
# sol.build_CP(X=sol_df.loc[29]['X'], XP=sol_df.loc[28]['XP'], CPbar=sol_df.loc[29]['CPbar'], WPbar=sol_df.loc[29]['WPbar'])
sol.compute_V(X=sol_df.loc[29]['X'], XP=sol_df.loc[29]['XP'], CPbar=sol_df.loc[29]['CPbar'], WPbar=sol_df.loc[29]['WPbar'])

In [ ]:
sol.build_VF(X=427.8222)

In [ ]:
sol = solutions[8] 
# sol = so_sol

In [ ]:
N, beta, g, a_bar = sol._unpack_params()
X = sol.erm.xgrid
VF = sol.VF
XP = sol.XP
Vp = sol.build_VF(XP)
CPbar = sol.CPbar
WPbar = sol.WPbar
CP = sol.build_CP(X=X, XP=XP, Vp=Vp, CPbar=CPbar, WPbar=WPbar)
WP = sol.build_WP(X=X, XP=XP, CP=CP, CPbar=CPbar, WPbar=WPbar)
# CP = sol.build_CP(X=X, XP=XP, Vp=Vp)
# WP = sol.build_WP(X=X, XP=XP, CP=CP)

In [ ]:
sol_df = pd.DataFrame({
    'X': X,
    'X/N': X/N,
    'XP': XP,
    'VF': VF,
    'CP': CP,
    'WP': WP,
    'CPbar': CPbar,
    'WPbar': WPbar,
    'CP + WP': CP + WP,
    'amt. spent': CP + WP + (N-1)*(CPbar + WPbar),
    'CP/WP': CP/WP,
    'X_dot': XP-X,
    'X_dot/X': (XP-X)/X,
})
sol_df.style

### Simulation Results:

We can scale the policy function by a factor of $ s $ to see if perturbations can get better results. Thus, ideally the best results should come when $ s = 1 $. Results of the perturbations are shown below.


In [ ]:
T = 100
df_sim = pd.DataFrame({
    'X/N': np.geomspace(10,5e3, 20),
    'T': T
})

df_sim_res1 = df_sim.apply(
    lambda x: list(sol.simulate(N*x['X/N'],x['T'],s=1)) 
    + [sol.simulate(N*x['X/N'],x['T'],s=0.9)[0]]
    + [sol.simulate(N*x['X/N'],x['T'],s=0.95)[0]]
    + [sol.simulate(N*x['X/N'],x['T'],s=0.99)[0]]
    + [sol.simulate(N*x['X/N'],x['T'],s=1.01)[0]]
    + [sol.simulate(N*x['X/N'],x['T'],s=1.05)[0]]
    + [sol.simulate(N*x['X/N'],x['T'],s=1.1)[0]]
    , axis=1
    , result_type='expand'
)
df_sim_res1.columns=['val', 'psurvival', 'final_state', 'n_periods', 
                     's=0.9', 's=0.95','s=0.99','s=1.01','s=1.05','s=1.1']
df_sim = df_sim.merge(df_sim_res1, left_index=True, right_index=True)
df_sim = df_sim.assign(
    val_vs_best_diff=lambda x: x['val'] - x[
        ['val', 's=0.9', 's=0.95','s=0.99','s=1.01','s=1.05','s=1.1']
    ].max(axis=1)
)

df_sim.style

### Probability of surviving to T

In the paper, we explain why human civilization is currently at a level of $ \tilde{X}/N \approx 274 $. We get this result by assuming $ 1 $ unit of consumption is equivalent to the level of 'extreme poverty' set by the World Bank, and $ \tilde{X} $ being global aggregate wealth, which includes the stock of natural resources. With these two values, we compute that global aggregate wealth per person is about 274 times the level of annual consumption for a person at the extreme poverty threshold.  

Using this we can now find the probability of surviving to any period $ T $ at the social optimum.

In [ ]:
df_sim2 = pd.DataFrame({
    'X/N': 274,
    'T': [50, 100, 500]
})

df_sim_res2 = df_sim2.apply(
    lambda x: list(sol.simulate(N*x['X/N'],x['T'],s=1))
    , axis=1
    , result_type='expand'
)
df_sim_res2.columns=['val', 'psurvival', 'final_state', 'n_periods']
df_sim2 = df_sim2.merge(df_sim_res2, left_index=True, right_index=True).drop('n_periods', axis=1)
df_sim2.style